In [1]:
!pip install BitVector
!pip install paramiko
!pip install pyfinite

In [2]:
from BitVector.BitVector import *
import sys
import requests
import itertools
from pyfinite import ffield
from pyfinite import genericmatrix
import re
import paramiko
from tqdm import tqdm


In [3]:
# some variables and constants

MOD             = BitVector(bitstring='10000011')
EXPO            = 7
twoPower        = []
matrixA         = []
vectorE         = []
InverseOfI      = [0, 1, 64, 85, 32, 51, 106, 109, 16, 113, 89, 104, 53, 88, 118, 17, 8, 15, 120, 107, 108, 121, 52, 116, 90, 61, 44, 80, 59, 92, 72, 41, 4, 77, 71, 98, 60, 103, 117, 114, 54, 31, 124, 65, 26, 48, 58, 100, 45, 70, 94, 5, 22, 12, 40, 97, 93, 78, 46, 28, 36, 25, 84, 125, 2, 43, 102, 91, 99, 81, 49, 34, 30, 87, 115, 105, 122, 33, 57, 82, 27, 69, 79, 101, 62, 3, 96, 73, 13, 10, 24, 67, 29, 56, 50, 123, 86, 55, 35, 68, 47, 83, 66, 37, 11, 75, 6, 19, 20, 7, 112, 119, 110, 9, 39, 74, 23, 38, 14, 111, 18, 21, 76, 95, 42, 63, 126, 1]



In [4]:
#string to bit vector
def strToBitV(str):
    bitV = []
    for i in range(0, len(str), 2):
        x1  = ord(str[i]) - ord('f')
        x2 = ord(str[i+1]) - ord('f')
        num = (x1 << 4) + x2
        bitV.append(BitVector(intVal = num, size = EXPO))
    return bitV


#  list of numbers to string letter

def listToStr(l):
    stri = ""
    for i in range(len(l)):
        x2 = chr(ord('f') + (l[i] & 0b1111))
        x1  = chr(ord('f') + ((l[i] & 0b1110000) >> 4))
        stri = stri + x1 + x2
    return stri


# creating bit vector
def bVector(num):
    return BitVector(intVal = num, size = EXPO)


# Calculate bitV to the power 
def expBitV(bitV, power):
    if power == 0:
        return bVector(1)
    elif power == 1:
        return bitV
    elif power % 2 == 0:
        half = expBitV(bitV, power >> 1)
        return bitVMultiply(half, half)
    else:
        half = expBitV(bitV, power >> 1)
        return bitVMultiply(bitVMultiply(half, half), bitV)


In [5]:
# connecting to server
def SSH_Connect(Inputs):
  Outputs = []
  hostname = "65.0.124.36"
  myuser = "student"
  mySSHK = "/content/ssh-keygen"


  sshcon = paramiko.SSHClient()
  sshcon.set_missing_host_key_policy(paramiko.AutoAddPolicy())
  sshcon.connect(hostname, username = myuser, key_filename = mySSHK)

  channel = sshcon.invoke_shell()
  stdin = channel.makefile('wb')
  stdout = channel.makefile('rb')

  stdin.write('''
  GrepTribe
  ''')

  stdin.write('''
  hello_bmro!
  ''')

  stdin.write('''
  5
  ''')

  stdin.write('''
  go
  ''')

  stdin.write('''
  wave
  ''')

  stdin.write('''
  dive
  ''')

  stdin.write('''
  go
  ''')

  stdin.write('''
  read
  ''')
  
  flag = True
  Outputs = []
  (stdout.read(4510).decode().splitlines()[-5:])
  
  for inputs in tqdm(Inputs):
    stdin.write(f'''
    {inputs}
    ''')
    (stdout.read(63+28+5+len(inputs)).decode().splitlines())
    Outputs.append(re.sub('\s+', '', stdout.read(30+((len(inputs)-1)//16+1)*16).decode().splitlines()[1]))
    stdin.write('''
    c
    ''')
    (stdout.read(62).decode().splitlines())

  stdin.close()
  stdout.close()

  return Outputs


In [6]:
#  multiply 2 bit vector

def bitVMultiply(bitV1, bitV2):
    return bitV1.gf_multiply_modular(bitV2, MOD, EXPO)

# add 2 bit vector

def bitVAdd(bitV1, bitV2):
    summed_value = int(str(bitV1), 2) ^ int(str(bitV2), 2)
    return bVector(summed_value)

# output at required index for given keys and input
def outputAtIndex(inp, outputIndex, A, E):
    j = outputIndex
    for t in range(8):
        if inp[t] != 0:
            i = t
            value = bVector(inp[t])
            break
    sum = bVector(0)
    for k in range(i,j+1):
        sum = bitVAdd(sum, bitVMultiply(expBitV(value, E[i]*E[k]), bitVMultiply(A[j][k], expBitV(A[k][i], E[k]))))
    return expBitV(sum, E[j]) 

# printing keys
def printKeys():
    for i in range(8):
        for j in range(8):
            print("%3d " % (int(str(matrixA[i][j]),2)), end=" ")
        print("E: %3d " % (vectorE[i]))
    print(" ")

In [7]:
# breaking cipher

def break_cipher():
    for i in range(128):
        twoPower.append(expBitV(bVector(2), i*i*i)) # 2^(i*i*i)
    for i in range(8):
        vectorE.append(0)
        matrixA.append([bVector(0)]*8)

    e_a = [[] for i in range(8)]
    for i in [7, 6, 5, 4, 3, 2, 1, 0]:
        cipherOne   = strToBitV("".join(SSH_Connect([listToStr([0]*i + [1] + [0]*(7-i))])))
        cipherTwo   = strToBitV("".join(SSH_Connect([listToStr([0]*i + [2] + [0]*(7-i))])))
        # print(cipherOne)
        #Get diagonal element
        for m in range(128):
            if bitVMultiply(cipherOne[i], twoPower[m]) == cipherTwo[i]:
                # vectorE[i].append(m)
                a_exponent = m*(m+1)
                for n in range(128):
                    if expBitV(bVector(n), a_exponent) == cipherOne[i]:
                        # matrixA[i][i].append(bVector(n))
                        e_a[i].append([m, bVector(n)])

    ai_i1_list = []
    for i in range(1,8):
        cipherOne   = strToBitV("".join(SSH_Connect([listToStr([0]*(i-1) + [1] + [0]*(8-i))])))
        cipherTwo   = strToBitV("".join(SSH_Connect([listToStr([0]*(i-1) + [2] + [0]*(8-i))])))
        counter = 0
        e_a_currPoss = []
        e_a_prevPoss = []
        flag = 0
        for m in e_a[i]:
            for n in e_a[i-1]:
                e7  = m[0]
                a77 = m[1]
                e6  = n[0]
                a66 = n[1]
                a66_e6 = expBitV(a66, e6)
                two_e6_e6 = expBitV(bVector(2), e6*e6)
                two_e6_e7 = expBitV(bVector(2), e6*e7)
                for t in range(128):
                    a76 = bVector(t)
                    if expBitV(bitVAdd(bitVMultiply(a76, a66_e6), bitVMultiply(a77, expBitV(a76, e7))), e7) == cipherOne[i] and expBitV(bitVAdd(bitVMultiply(bitVMultiply(two_e6_e6, a66_e6), a76), bitVMultiply(two_e6_e7, bitVMultiply(expBitV(a76, e7), a77))) ,e7) == cipherTwo[i]:
                        if i == 7:
                            ai_i1_list.append([a76, a77, e7])
                        if m not in e_a_currPoss:
                            e_a_currPoss.append(m)
                        if n not in e_a_prevPoss:
                            e_a_prevPoss.append(n)
                        counter = counter + 1
                        flag = 1
                        matrixA[i][i-1] = a76
            if flag == 1:
                break
        e_a[i] = e_a_currPoss
        e_a[i-1] = e_a_prevPoss
    for i in range(8):
        matrixA[i][i] = e_a[i][0][1]
        vectorE[i]    = e_a[i][0][0]
    printKeys()

    for p in range(2,8):
        for k in range(p,8):
            saved_values = []
            cipherOne   = strToBitV("".join(SSH_Connect([listToStr([0]*(k-p) + [1] + [0]*(7-k+p))])))
            cipherTwo   = strToBitV("".join(SSH_Connect([listToStr([0]*(k-p) + [2] + [0]*(7-k+p))])))
            
            for i in range(128):
                matrixA[k][k-p] = bVector(i)
                if outputAtIndex([0]*(k-p) + [1] + [0]*(7-k+p), k, matrixA, vectorE) == cipherOne[k] and outputAtIndex([0]*(k-p) + [2] + [0]*(7-k+p), k, matrixA, vectorE) == cipherTwo[k]:
                    saved_values.append(i)
            if(len(saved_values)>1):
              print("For cell " ,k,',',k-p,"possible values are ", saved_values) 
            matrixA[k][k-p] = bVector(saved_values[0])
        printKeys()

break_cipher()

100%|██████████| 1/1 [00:00<00:00,  2.31it/s]


 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
  0   27   43    0    0    0    0    0  E:  43 
  0    0    6   12    0    0    0    0  E:  81 
  0    0    0  112  112    0    0    0  E:  91 
  0    0    0    0   99   11    0    0  E:  46 
  0    0    0    0    0   95   27    0  E:  21 
  0    0    0    0    0    0    5   38  E:  24 
 


100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
  0   23    6   12    0    0    0    0  E:  81 
  0    0    2  112  112    0    0    0  E:  91 
  0    0    0   41   99   11    0    0  E:  46 
  0    0    0    0    5   95   27    0  E:  21 
  0    0    0    0    0   66    5   38  E:  24 
 


100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


...

 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
122   23    6   12    0    0    0    0  E:  81 
 96   60    2  112  112    0    0    0  E:  91 
 25   41   19   41   99   11    0    0  E:  46 
  0  125    1   98    5   95   27    0  E:  21 
  0    0   72   24   18   66    5   38  E:  24 
 


100%|██████████| 1/1 [00:00<00:00,  2.37it/s]


 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
122   23    6   12    0    0    0    0  E:  81 
 96   60    2  112  112    0    0    0  E:  91 
 25   41   19   41   99   11    0    0  E:  46 
 22  125    1   98    5   95   27    0  E:  21 
  0   12   72   24   18   66    5   38  E:  24 
 


100%|██████████| 1/1 [00:00<00:00,  2.36it/s]


For cell  7 , 0 possible values are  [56, 70, 82]
 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
122   23    6   12    0    0    0    0  E:  81 
 96   60    2  112  112    0    0    0  E:  91 
 25   41   19   41   99   11    0    0  E:  46 
 22  125    1   98    5   95   27    0  E:  21 
 56   12   72   24   18   66    5   38  E:  24 
 


In [8]:
# finding inverse of A
def invertA():
    A_Inverse = []
    field = ffield.FField(7,gen=0x83)
    matrix = genericmatrix.GenericMatrix(size=(8,8), zeroElement=0, identityElement=1, add=field.Add, mul=field.Multiply, sub=field.Subtract, div=field.Divide)
    matrix.SetRow(0,[int(str(i), 2) for i in matrixA[0]])
    matrix.SetRow(1,[int(str(i), 2) for i in matrixA[1]])
    matrix.SetRow(2,[int(str(i), 2) for i in matrixA[2]])
    matrix.SetRow(3,[int(str(i), 2) for i in matrixA[3]])
    matrix.SetRow(4,[int(str(i), 2) for i in matrixA[4]])
    matrix.SetRow(5,[int(str(i), 2) for i in matrixA[5]])
    matrix.SetRow(6,[int(str(i), 2) for i in matrixA[6]])
    matrix.SetRow(7,[int(str(i), 2) for i in matrixA[7]])
    inverse = matrix.Inverse()
    for rownum in range(8):
        row = [bVector(i) for i in inverse.GetRow(rownum)]
        A_Inverse.append(row)
    return A_Inverse

#finding output of inverse E
def invertE(vec):
    result = []
    for i in range(8):
        v = vec[i]
        e = vectorE[i]
        inv =  InverseOfI[e]
        result.append(expBitV(v, inv))
    return result

#finding output of inverse A
def OutputInverseA(vec):
    result = []
    for i in range(8):
        sum = bVector(0)
        for j in range(8):
            sum = bitVAdd(sum, bitVMultiply(A_Inverse[i][j], vec[j]))
        result.append(sum)
    return result

# decrypt output to find  input
def decrypt(string):
    vec = strToBitV(string)
    vec = invertE(vec)
    vec = OutputInverseA(vec)
    vec = invertE(vec)
    vec = OutputInverseA(vec)
    vec = invertE(vec)
    return [int(str(i), 2) for i in vec]

In [9]:
# finding passwords
coded_password = "jghrlglshrhkhqmggrjkiihqgsgmjriq"
possiblePassword = []
p1 = coded_password[:16]
p2 = coded_password[16:] 
for val in [56, 70, 82]:
  matrixA[7][0] = bVector(val)
  A_Inverse = invertA()
  passwordAscii = decrypt(p1)+decrypt(p2)
  password = ""
  for i in passwordAscii:
      password = password + chr(i)
  possiblePassword.append(password)
  printKeys()
  print(password)
print("possible passwords are : ", possiblePassword)
print("After trying all three password we get the real password i.e. :- ",'xjtwcsnucg')

 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
122   23    6   12    0    0    0    0  E:  81 
 96   60    2  112  112    0    0    0  E:  91 
 25   41   19   41   99   11    0    0  E:  46 
 22  125    1   98    5   95   27    0  E:  21 
 56   12   72   24   18   66    5   38  E:  24 
 
xjtwcsncg00000f
 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
122   23    6   12    0    0    0    0  E:  81 
 96   60    2  112  112    0    0    0  E:  91 
 25   41   19   41   99   11    0    0  E:  46 
 22  125    1   98    5   95   27    0  E:  21 
 70   12   72   24   18   66    5   38  E:  24 
 
xjtwcsnucg000000
 84    0    0    0    0    0    0    0  E:  24 
118   70    0    0    0    0    0    0  E: 105 
 27   27   43    0    0    0    0    0  E:  43 
122   23    6   12    0    0    0    0  E:  81 
 9